In [4]:
import io
import torch
from torch.utils.data import IterableDataset
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
#поїхали. мій час. задоволення.
train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
counter = Counter()
for line in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter)

In [2]:
def data_process(raw_text_iter):
  data = [torch.tensor([vocab[token] for token in tokenizer(item)],
                       dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_iter, val_iter, test_iter = WikiText2()
print(type(train_iter))
train_data = data_process(train_iter)

<class 'torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe'>


In [40]:
import csv
import random

class ShardingFilterIterDataPipe:
    def __init__(self, shard_index, shard_num, batch_size):
        self.shard_index = shard_index
        self.shard_num = shard_num
        self.batch_size = batch_size
        self.rows = []
        self.read_csv()

    def read_csv(self):
        """Reads the dataset.csv file and stores all rows in self.rows"""
        with open('dataset.csv', newline='') as file:
            reader = csv.reader(file)
            for row in reader:
                self.rows.append(row)
    
    def __iter__(self):
        """Return an iterator over the rows"""
        random.shuffle(self.rows)
        for i in range(0, len(self.rows), self.batch_size):
            yield self.rows[i:i + self.batch_size]

    def __len__(self):
        """Return the number of batches in the data pipe"""
        return len(self.rows) // self.batch_size
data_pipe = ShardingFilterIterDataPipe(0, 1, 2)
for i,batch in enumerate(data_pipe):
    print(i,batch)





0 [['False', '12', '12', '11', '10', '4'], ['True', '5', '3', '3', '-1', '-1']]
1 [['True', '4', '10', '2', '-1', '-1'], ['False', '7', '3', '4', '5', '4']]
2 [['False', '12', '11', '-1', '-1', '-1'], ['True', '6', '7', '3', '-1', '-1']]
3 [['False', '3', '12', '13', '-1', '-1'], ['False', '7', '10', '-1', '-1', '-1']]
4 [['True', '7', '3', '5', '7', '2'], ['False', '9', '2', '1', '1', '-1']]
5 [['False', '13', '5', '9', '7', '4'], ['True', '3', '6', '7', '2', '-1']]
6 [['False', '10', '-1', '-1', '-1', '-1'], ['True', '1', '5', '1', '-1', '-1']]
7 [['True', '2', '8', '2', '12', '5'], ['False', '10', '12', '-1', '-1', '-1']]
8 [['True', '11', '1', '-1', '-1', '-1'], ['False', '5', '9', '-1', '-1', '-1']]
9 [['True', '5', '7', '2', '1', '-1'], ['False', '3', '9', '6', '6', '12']]
10 [['True', '8', '2', '5', '3', '8'], ['False', '4', '6', '1', '-1', '-1']]
11 [['True', '5', '2', '8', '2', '-1'], ['False', '7', '11', '-1', '-1', '-1']]
12 [['False', '10', '6', '8', '7', '12'], ['False', '